In [87]:
import numpy as np
import math
import xlwings as xw

In [88]:
#Variables
S = 100  #Stock price at t=0
r = 0.05  #Risk free rate
dt = .001  #Timestep size
T = 1  #time to expiry in years
sigma = .250
E = 110  #Strike Price


length = T/dt
u = math.exp( sigma * math.sqrt(dt))
d = math.exp(- sigma * math.sqrt(dt))
a = math.exp( r * dt)
p = (a - d) / (u - d)

u,d,a,p

(1.0079370266644199,
 0.9921254736611017,
 1.0000500012500209,
 0.5011859105336545)

In [89]:
#Create Empty Numpy Arrays to hold the data
step_matrix = np.zeros((int(length), int(length+1)))
call_option_value_matrix = np.zeros((int(length+1), int(length+1)))
put_option_value_matrix = np.zeros((int(length+1), int(length+1)))

stock_value_matrix = np.zeros((int(length+1), int(length+1)))
#Set initial value for the stock
stock_value_matrix[0,0] = S

In [90]:
#Create binomial tree of the pricing of the underlying
for x in range(1,int(length+1)):
    stock_value_matrix[x,0] = stock_value_matrix[x-1,0]*(d)
    for y in range(1,int(length+1)):
        stock_value_matrix[x,y] = stock_value_matrix[x-1,y-1]*(u)
#xw.view(value_matrix)
#stock_value_matrix

In [91]:
#Create Option Valuation Tree Call Option
for x in range(1, int(length+1)):
    for y in range(0, int(length+1)):
        if stock_value_matrix[-x , y]-E > 0:
            call_option_value_matrix[-x, y] = stock_value_matrix[-x , y] - E
        else:
            call_option_value_matrix[-x , y] = 0
            
#Create Option Valuation Tree Put Option
for x in range(1, int(length+1)):
    for y in range(0, int(length+1)):
        if E - stock_value_matrix[-x , y] > 0:
            put_option_value_matrix[-x, y] = E - stock_value_matrix[-x , y]
        else:
            put_option_value_matrix[-x , y] = 0

In [92]:
call_payoff_matrix = np.zeros((int(length+1), int(length+1)))
call_payoff_matrix[-1] = call_option_value_matrix[-1]
count = 0
for x in range(1,int(length+1)):
    count += 1
    for y in range(count, int(length+1)):
        call_payoff_matrix[-x-1,-y-1] = math.exp(-r*dt)*((p*call_payoff_matrix[-x,-y])+((1-p)*call_payoff_matrix[-x,-y-1]))

put_payoff_matrix = np.zeros((int(length+1), int(length+1)))
put_payoff_matrix[-1] = put_option_value_matrix[-1]
count = 0
for x in range(1,int(length+1)):
    count += 1
    for y in range(count, int(length+1)):
        put_payoff_matrix[-x-1,-y-1] = math.exp(-r*dt)*((p*put_payoff_matrix[-x,-y])+((1-p)*put_payoff_matrix[-x,-y-1]))

#xw.view(put_payoff_matrix)
#xw.view(call_payoff_matrix)
call_value = call_payoff_matrix[0,0]
put_value = put_payoff_matrix[0,0]

print("Value of Call Option Binomial = %.4f " %call_value)
print("Value of Put Option Binomial =  %.4f " %put_value)

Value of Call Option Binomial = 8.0241 
Value of Put Option Binomial =  12.6593 


In [93]:
#Module Imports
import scipy as sp
from scipy import stats

In [94]:
'''DEFINITION OF VARIABLES
    S0 - Stock Price at T=0
    E - Strike Price
    T - Time in Years
    R - Risk Free Rate
    SIGMA - Volatility
    DT - Time Step = T/N
    DF - Discount Factor = e^-RT
'''

S0 = S
E = E
T = T
R = r
SIGMA = sigma

In [95]:
'''BSM VANILLA EUROPEAN OPTION VALUE CALCULATION'''
def bsm_option_value(S0, E, T, R, SIGMA):   
    S0 = float(S0)
    d1 = (math.log(S0/E)+(R+0.5*SIGMA**2)*T)/(SIGMA*math.sqrt(T))
    d2 = d1-(SIGMA*math.sqrt(T))
    
    call_value = S0*stats.norm.cdf(d1,0,1) - E*math.exp(-R*T)*stats.norm.cdf(d2,0,1)
    
    put_value =  E*math.exp(-R*T)*stats.norm.cdf(-d2,0,1) - (S0*stats.norm.cdf(-d1,0,1))

    print("Value of Call Option BSM =  " + str(call_value))
    print("Value of Put Option BSM =  " + str(put_value))
    return

In [96]:
bsm_option_value(S0, E, T, R, SIGMA)

Value of Call Option BSM =  8.02638469385
Value of Put Option BSM =  12.6616213889
